# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations, changed sizes from 1500 to 15
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
print(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{url}appid={weather_api_key}&units={units}&q="
city_count = 0
lat=[]
lon=[]
max_temp=[]
hum=[]
cld_cov=[]
wnd_sp=[]
country=[]
datetime=[]

for city in cities:
    response = requests.get(query_url + city).json()
    city_count =city_count + 1
    try:     
        lat.append(response["coord"]["lat"])
        lon.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        hum.append(response["main"]["humidity"])
        cld_cov.append(response["clouds"]["all"])
        wnd_sp.append(response["wind"]["speed"])
        country.append(response["sys"]["country"])
        datetime.append(response["dt"])
        
        
    except:
        print("Error")
        pass
    print(f"City Number, Name: {city_count}, {city}")
#query_url

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_dict = {
    "Date-Time": datetime,
    "City": cities,
    "Country": country,
    "Lat": lat,
    "Lon": lon,
    "Max-Temp": max_temp,
    "Humidity": hum,
    "Cloud Cov": cld_cov,
    "Wind Sp": wnd_sp
}
weather_data = pd.DataFrame(weather_dict)
weather_data.to_csv("City_Weatherstats.csv")
weather_data

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humid_cities = weather_data.loc[:,"Humidity"]>100
humid_cities.value_counts()

In [ ]:
#  Get the indices of cities that have humidity over 100%.
hum_ind = humid_cities.index[humid_cities]
hum_ind

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = weather_data.drop(weather_data[weather_data["Humidity"]>100].index, inplace=False)
clean_city_data

In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv
clean_city_data.to_csv("City_Data.csv")

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
x_values = weather_data['Lat']
y_values = weather_data['Max-Temp']
plt.scatter(x_values,y_values)
plt.xlabel('Latitude')
plt.ylabel('Temperature (F)')
plt.title("Latitude vs. Temerature")
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
x_values = weather_data['Lat']
y_values = weather_data['Humidity']
plt.scatter(x_values,y_values)
plt.xlabel('Latitude')
plt.ylabel('Humidity (%)')
plt.title("Latitude vs. Humidity")
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
x_values = weather_data['Lat']
y_values = weather_data['Cloud Cov']
plt.scatter(x_values,y_values)
plt.xlabel('Latitude')
plt.ylabel('Cloud Coverage (%)')
plt.title("Latitude vs. Cloudiness")
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
x_values = weather_data['Lat']
y_values = weather_data['Wind Sp']
plt.scatter(x_values,y_values)
plt.xlabel('Latitude')
plt.ylabel('Wind Speed (mph)')
plt.title("Latitude vs. Wind Speed")
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northern_hem = weather_data.drop(weather_data[weather_data["Lat"]>0].index, inplace=True)

southern_hem = weather_data.drop(weather_data[weather_data["Lat"]<0].index, inplace=True)
#northern_hem.head()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = northern_hem['Lat']
y_values = northern_hem['Max-Temp']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Temperature')
plt.title("North Hem Max Temp (F) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = southern_hem['Lat']
y_values = southern_hem['Max-Temp']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Temperature')
plt.title("Southern Hem Max Temp (F) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_hem['Lat']
y_values = northern_hem['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title("North Hem Humidity (%) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_hem['Lat']
y_values = southern_hem['Humidity']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Humidity')
plt.title("Southern Hem Humidity (%) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = northern_hem['Lat']
y_values = northern_hem['Cloud Cov']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Cloud Cov')
plt.title("North Hem Cloud Coverage (%) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_hem['Lat']
y_values = southern_hem['Cloud Cov']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Cloud Cov')
plt.title("Southern Hem Cloud Cov (%) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = northern_hem['Lat']
y_values = northern_hem['Wind Sp']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.title("North Hem Wind Speed (mph) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = southern_hem['Lat']
y_values = southern_hem['Wind Sp']

(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))


plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Latitude')
plt.ylabel('Wind Speed')
plt.title("Southern Hem Wind Speed (mph) vs Latitude Linear Regressison")
plt.annotate(line_eq,(20,15),fontsize=15,color="red")



plt.show()